# LDA - RF Pipeline 

## Import necessary packages

In [1]:
import time
import optuna
import numpy as np
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import NMF
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
# from bitermplus import BTM

from dataset.dataset import Dataset
from constants import CLEANED_DATASET_PATH

## Constants

In [2]:
RANDOM_SEED = 0

## Load the Hate Speech Filipino dataset from Hugging Face

In [3]:
dataset = Dataset(full_data_path=CLEANED_DATASET_PATH,
                  from_scratch=False,
                  split_sizes=[10000, 4232, 4232])
dataset.build()

X_train = dataset.get_features(split_type="train")
Y_train = dataset.get_labels(split_type="train")
X_val = dataset.get_features(split_type="val")
Y_val = dataset.get_labels(split_type="val")
X_test = dataset.get_features(split_type="test")
Y_test = dataset.get_labels(split_type="test")

Data loaded from dataset/cleaned_dataset.pkl


In [4]:
# Re-split the dataset into training, validation, and test sets
# X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.2, random_state=RANDOM_SEED)
# X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=RANDOM_SEED)

## Classifier Only Pipeline

In [5]:
def create_objective(X_train, Y_train, X_test, Y_test, random_seed):
    def objective(trial):
        # Suggest values for the hyperparameters
        n_estimators = trial.suggest_int("n_estimators", 100, 1000)

        # Create the pipeline within the trial
        pipeline = Pipeline([
            ("vectorizer", CountVectorizer(max_df=0.95, min_df=2)),
            ("rf", RandomForestClassifier(n_estimators=n_estimators, random_state=random_seed))
        ])

        # Fit the pipeline
        pipeline.fit(X_train, Y_train)

        # Evaluate the pipeline
        Y_pred = pipeline.predict(X_test)
        accuracy = accuracy_score(Y_test, Y_pred)
        return accuracy

    return objective

In [6]:
# Create the objective function
objective = create_objective(X_train, Y_train, X_val, Y_val, RANDOM_SEED)

# Create an Optuna study object
study = optuna.create_study(direction="maximize", study_name=f"RF_Pipeline")

# Execute an optimization by running trials
study.optimize(objective, n_trials=30, n_jobs=-1)

best_rf_params = study.best_trial.params

# Best trial result
print(f"Best training accuracy: {study.best_trial.value}")
print(f"Best training hyperparameters: {best_rf_params}")

[I 2024-05-03 18:06:24,733] A new study created in memory with name: RF_Pipeline
[I 2024-05-03 18:06:42,924] Trial 6 finished with value: 0.7431474480151229 and parameters: {'n_estimators': 103}. Best is trial 6 with value: 0.7431474480151229.
[I 2024-05-03 18:07:39,179] Trial 3 finished with value: 0.7431474480151229 and parameters: {'n_estimators': 424}. Best is trial 6 with value: 0.7431474480151229.
[I 2024-05-03 18:07:45,989] Trial 4 finished with value: 0.7422022684310019 and parameters: {'n_estimators': 460}. Best is trial 6 with value: 0.7431474480151229.
[I 2024-05-03 18:07:51,381] Trial 11 finished with value: 0.7429111531190926 and parameters: {'n_estimators': 495}. Best is trial 6 with value: 0.7431474480151229.
[I 2024-05-03 18:07:51,601] Trial 8 finished with value: 0.7431474480151229 and parameters: {'n_estimators': 493}. Best is trial 6 with value: 0.7431474480151229.
[I 2024-05-03 18:07:55,063] Trial 9 finished with value: 0.7436200378071833 and parameters: {'n_estimat

Best training accuracy: 0.7474007561436673
Best training hyperparameters: {'n_estimators': 640}


In [11]:
# Create and fit the pipeline
pipeline = Pipeline([
    ("vectorizer", CountVectorizer(max_df=0.95, min_df=2)),
    ("rf", RandomForestClassifier(n_estimators=best_rf_params["n_estimators"], 
                                  random_state=RANDOM_SEED))
])

# Fit the pipeline
pipeline.fit(X_train, Y_train)

# Evaluate the pipeline
Y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred)

# Best testing result
print(f"Best test accuracy: {accuracy}")
print(f"Best test F1-score: {f1}")
print(f"Best test hyperparameters: {study.best_trial.params}")

Best test accuracy: 0.7372400756143668
Best test F1-score: 0.7031500266951415
Best test hyperparameters: {'n_topics': 236}


## NMF-RF Pipeline

In [8]:
def create_objective(X_train, Y_train, X_test, Y_test, best_rf_params, random_seed):
    def objective(trial):
        # Suggest values for the hyperparameters
        n_topics = trial.suggest_int("n_topics", 50, 300)

        # Create the pipeline within the trial
        pipeline = Pipeline([
            ("vectorizer", CountVectorizer(max_df=0.95, min_df=2)),
            ("nmf", NMF(n_components=n_topics, random_state=random_seed)),
            ("rf", RandomForestClassifier(n_estimators=best_rf_params["n_estimators"], 
                                          random_state=random_seed))
        ])

        # Fit the pipeline
        pipeline.fit(X_train, Y_train)

        # Evaluate the pipeline
        Y_pred = pipeline.predict(X_test)
        accuracy = accuracy_score(Y_test, Y_pred)
        return accuracy

    return objective

In [9]:
# Create the objective function
objective = create_objective(X_train, Y_train, X_val, Y_val, best_rf_params, RANDOM_SEED)

# Create an Optuna study object
study = optuna.create_study(direction="maximize", study_name=f"NMF_RF_Pipeline")

# Execute an optimization by running trials
study.optimize(objective, n_trials=30, n_jobs=-1)

best_nmf_params = study.best_trial.params

# Best trial result
print(f"Best training accuracy: {study.best_trial.value}")
print(f"Best training hyperparameters: {best_nmf_params}")

[I 2024-05-03 18:15:46,904] A new study created in memory with name: NMF_RF_Pipeline
[I 2024-05-03 18:16:27,394] Trial 4 finished with value: 0.7003780718336484 and parameters: {'n_topics': 52}. Best is trial 4 with value: 0.7003780718336484.
[I 2024-05-03 18:16:36,485] Trial 5 finished with value: 0.7053402646502835 and parameters: {'n_topics': 57}. Best is trial 5 with value: 0.7053402646502835.
[I 2024-05-03 18:17:01,027] Trial 2 finished with value: 0.7214083175803403 and parameters: {'n_topics': 93}. Best is trial 2 with value: 0.7214083175803403.
[I 2024-05-03 18:17:15,293] Trial 0 finished with value: 0.7190453686200378 and parameters: {'n_topics': 95}. Best is trial 2 with value: 0.7214083175803403.
[I 2024-05-03 18:17:24,116] Trial 9 finished with value: 0.7244801512287334 and parameters: {'n_topics': 104}. Best is trial 9 with value: 0.7244801512287334.
[I 2024-05-03 18:17:36,635] Trial 13 finished with value: 0.7091209829867675 and parameters: {'n_topics': 62}. Best is trial

Best training accuracy: 0.745274102079395
Best training hyperparameters: {'n_topics': 236}


In [10]:
# Create and fit the pipeline
pipeline = Pipeline([
    ("vectorizer", CountVectorizer(max_df=0.95, min_df=2)),
    ("nmf", NMF(n_components=best_nmf_params["n_topics"], 
                random_state=RANDOM_SEED)),
    ("rf", RandomForestClassifier(n_estimators=best_rf_params["n_estimators"], 
                                  random_state=RANDOM_SEED))
])

# Fit the pipeline
pipeline.fit(X_train, Y_train)

# Evaluate the pipeline
Y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred)

# Best testing result
print(f"Best test accuracy: {accuracy}")
print(f"Best test F1-score: {f1}")
print(f"Best test hyperparameters: {study.best_trial.params}")

c:\Users\lendl\Documents\hate-speech-detection\venv\lib\site-packages\sklearn\decomposition\_nmf.py:1770: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


Best test accuracy: 0.7268431001890359
Best test F1-score: 0.7025218733916624
Best test hyperparameters: {'n_topics': 236}
